In [1]:
import kagglehub, os

# Download latest version
path = kagglehub.dataset_download("apollo2506/eurosat-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/apollo2506/eurosat-dataset/versions/6


In [1]:
import torchvision.transforms as transforms

import src

from src.runner import Runner
import os


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

loading_cfg = {
    'batch_size': 1024,
    'num_workers': 4,
}

data_cfg = {
    'type': 'EuroSATDataset'
}

optim_cfg = {
    'type': 'Adam',
    'lr': 0.001,
    'weight_decay': 1e-4,
}


model_cfg = {
    'type': 'EuroSATModel',
    'backbone_cfg': {
        'type': 'ResNet',
        'idims': 3,
        'odims': 64,
        'base_dims': 12,
        'arch': [2,2,2,2],
        'dropout': 0.2,
    },
    'head_cfg': {
        'type': 'FFN',
        'idims': 64,
        'odims': 10,  # EuroSAT has 10 classes
        'hidden_dims': 1024,
        'nlayers':6,
        'dropout': 0.2,
    }
}

runner = Runner(model=model_cfg, dataloader_cfg=loading_cfg, dataset=data_cfg, optim=optim_cfg, device='cuda:2', work_dir='results/eurosat')
runner.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

No seed provided, using initial seed: 1971015095
     Model Parameters Summary     
----------------------------------
Trainable params     | 5,857,878
Non-trainable params |         0
Total params         | 5,857,878
- Backbone params    | 1,582,668
- Head params        | 4,275,210
Epoch 001/100 | Iter 19/19 | [██████████] |  | loss: 1.3130 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>>>>>>] |  100.00% | val_loss: 2.8764 --> F1: 0.1023 | mAP: 0.2196
Model saved to results/eurosat/run_20250612-220000/best_model.pth
Best model saved at epoch 1 with val_loss 2.2235
Epoch 002/100 | Iter 19/19 | [██████████] |  | loss: 1.0885 | lr: 1.0000e-03
Evaluating | Iter 0201/5400 | [----------] |  3.72% | val_loss: 2.3240

KeyboardInterrupt: 

In [1]:
data_cfg = {
    'type': 'ImageNetDataset',
}

import src
from src.build.registry import build_module, DATASETS

dataset = build_module(data_cfg, DATASETS)




In [ ]:
loading_cfg = dict(
    batch_size=1024,
    num_workers=4,
)

eurosat_cfg = dict(
    type='EuroSATDataset',
    transform=[
        dict(type='Resize', size=(128, 128)),
        dict(type='ToTensor'),
    ]
)

imagenet_cfg = dict(
    type='ImageNetDataset',
    transform=[
        dict(type='Resize', size=(128, 128)),
        dict(type='ToTensor'),
    ]
)

optim_cfg = dict(
    type='Adam',
    lr=0.001,
    weight_decay=1e-4,
)

backbone_cfg = dict(
    type='ResNet',
    idims=3,
    odims=64,
    base_dims=12,
    arch=[2, 2, 2, 2],
    dropout=0.2,
)

In [ ]:
import torchvision.transforms as transforms

import src

from src.runner import Runner



# -----------------------------------
# Vanilla EuroSAT model configuration
# -----------------------------------

eurosat_model_cfg = dict(
    type='EuroSATModel',
    backbone_cfg=backbone_cfg,
    head_cfg=dict(
        type='FFN',
        idims=64,
        odims=10,  # EuroSAT has 10 classes
        hidden_dims=1024,
        nlayers=6,
        dropout=0.2,
    )
)

eurosat_vanilla = Runner(model=eurosat_model_cfg, dataloader_cfg=loading_cfg, dataset=eurosat_cfg, optim=optim_cfg, device='cuda:2', work_dir='results/eurosat_vanilla')
eurosat_vanilla.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

# -----------------------------------
# Tiny ImageNet model configuration
# -----------------------------------
tiny_imnet_cfg = dict(
    type='EuroSATModel',
    backbone_cfg=backbone_cfg,
    head_cfg=dict(
        type='FFN',
        idims=64,
        odims=200,  # Tiny ImageNet has 200 classes
        hidden_dims=1024,
        nlayers=6,
        dropout=0.2,
    )
)

tiny_imnet = Runner(model=tiny_imnet_cfg, dataloader_cfg=loading_cfg, dataset=imagenet_cfg, optim=optim_cfg, device='cuda:2', work_dir='results/tiny_imnet')
tiny_imnet.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)


# -----------------------------------
# TFL EuroSAT model configuration
# -----------------------------------
tfl_eurosat_model_cfg = dict(
    type='EuroSATModel',
    backbone_cfg=backbone_cfg,
    head_cfg=dict(
        type='FFN',
        idims=64,
        odims=10,  # EuroSAT has 10 classes
        hidden_dims=1024,
        nlayers=6,
        dropout=0.2,
    ),
    ckpt=dict(
        path = tiny_imnet.best_model_path,
        load_head=False,
        load_backbone=True,
        strict=True,
    )
)

tfl_eurosat = Runner(model=tfl_eurosat_model_cfg, dataloader_cfg=loading_cfg, dataset=eurosat_cfg, optim=optim_cfg, device='cuda:2', work_dir='results/tfl_eurosat')
tfl_eurosat.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

No seed provided, using initial seed: 951366024
     Model Parameters Summary     
----------------------------------
Trainable params     | 6,052,628
Non-trainable params |         0
Total params         | 6,052,628
- Backbone params    | 1,582,668
- Head params        | 4,469,960
Epoch 001/100 | Iter 98/98 | [██████████] |  | loss: 4.9387 | lr: 1.0000e-03
Evaluating | Iter 10000/10000 | [>>>>>>>>>>] |  100.00% | val_loss: 4.9496 --> F1: 0.0038 | mAP: 0.0196
Model saved to results/imagenet/run_20250612-152058/best_model.pth
Best model saved at epoch 1 with val_loss 5.2939
Epoch 002/100 | Iter 98/98 | [██████████] |  | loss: 4.6264 | lr: 1.0000e-03
Evaluating | Iter 10000/10000 | [>>>>>>>>>>] |  100.00% | val_loss: 4.6816 --> F1: 0.0169 | mAP: 0.0325
Model saved to results/imagenet/run_20250612-152058/best_model.pth
Best model saved at epoch 2 with val_loss 5.2881
Epoch 003/100 | Iter 98/98 | [██████████] |  | loss: 4.4499 | lr: 1.0000e-03
Evaluating | Iter 10000/10000 | [>>>>>>>>>>] |

{'train_loss': [5.094094870041828,
  4.806673035329702,
  4.586937018803188,
  4.346366293576299,
  4.0821913821356635,
  3.8712936591128915,
  3.7060899101957983,
  3.556691439784303,
  3.4093872138432095,
  3.278604208206644,
  3.142603443593395,
  3.0176034411605523,
  2.89690422281927,
  2.782175973970063,
  2.67751898084368,
  2.5509630733606765,
  2.4380312355197207,
  2.322100651507475,
  2.2175288954559638,
  2.084393141221027,
  1.9632532268154377,
  1.8317359114179805,
  1.7253172044851341,
  1.6063980241211093,
  1.499154152918835,
  1.4132710366832966,
  1.2965002789789317],
 'val_loss': [5.2938995361328125,
  5.288149833679199,
  5.281394004821777,
  5.274620532989502,
  5.25952672958374,
  5.2568206787109375,
  5.244368553161621,
  5.222179412841797,
  5.216206073760986,
  5.196222305297852,
  5.188464641571045,
  5.182450294494629,
  5.16572904586792,
  5.1597676277160645,
  5.160167217254639,
  5.150108814239502,
  5.144759654998779,
  5.140834808349609,
  5.12827730178

In [6]:
import torch
transfer_model = torch.load('/mmdetection3d/PRIVATE/MLSP/results/eurosat/run_20250612-220000/best_model.pth', weights_only=False)

In [ ]:
from src.build.registry import build_module, MODULES



model_cfg = {
    'type': 'EuroSATModel',
    'backbone_cfg': {
        'type': 'ResNet',
        'idims': 3,
        'odims': 64,
        'base_dims': 12,
        'arch': [2,2,2,2],
        'dropout': 0.2,
    },
    'head_cfg': {
        'type': 'FFN',
        'idims': 64,
        'odims': 10,  # EuroSAT has 10 classes
        'hidden_dims': 1024,
        'nlayers':6,
        'dropout': 0.2,
    },
    'ckpt':{
        'path': '/mmdetection3d/PRIVATE/MLSP/results/eurosat/run_20250612-220000/best_model.pth',
        'load_head': False,
        'load_backbone': True,
        'strict': True,
    }
}

model = build_module(model_cfg, MODULES)

In [10]:
# load only the backbone weights
model.backbone.load_state_dict(transfer_model['backbone'], strict=True)

<All keys matched successfully>

In [11]:
runner_tfl = Runner(model=model, dataloader_cfg=loading_cfg, dataset=data_cfg, optim=optim_cfg, device='cuda:2', work_dir='results/imagenet')
runner_tfl.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

No seed provided, using initial seed: 1971015095
     Model Parameters Summary     
----------------------------------
Trainable params     | 5,857,878
Non-trainable params |         0
Total params         | 5,857,878
- Backbone params    | 1,582,668
- Head params        | 4,275,210
Epoch 001/100 | Iter 19/19 | [██████████] |  | loss: 1.3158 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>>>>>>] |  100.00% | val_loss: 2.2864 --> F1: 0.0880 | mAP: 0.2361
Model saved to results/imagenet/run_20250612-220544/best_model.pth
Best model saved at epoch 1 with val_loss 2.2327
Epoch 002/100 | Iter 19/19 | [██████████] |  | loss: 1.0969 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>>>>>>] |  100.00% | val_loss: 1.3027 --> F1: 0.4366 | mAP: 0.5032
Model saved to results/imagenet/run_20250612-220544/best_model.pth
Best model saved at epoch 2 with val_loss 2.0395
Epoch 003/100 | Iter 19/19 | [██████████] |  | loss: 0.9099 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>>>>>>] |  100

Exception in thread Thread-17 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/torch/multiprocessi

KeyboardInterrupt: 